In [ ]:
# A* algorithm
from collections import defaultdict

class LinkedListNode:
	def __init__(self, item, value):
		self.item = item
		self.value = value
		self.next_node = None
		self.previous_node = None

	def get_item(self):
		return self.item

	def get_value(self):
		return self.value

	def get_next_node(self):
		return self.next_node

	def get_previous_node(self):
		return self.previous_node

	def set_next_node(self, next_node):
		self.next_node = next_node

	def set_previous_node(self, previous_node):
		self.previous_node = previous_node

class SortedLinkedList:
	def __init__(self):
		self.first_node = None
		self.last_node = None
		self.item_to_node = {}
		self.first_node_at_value = {}

	def remove_node(self, node):
		node_before = node.get_previous_node()
		node_after = node.get_next_node()

		if node_before is None:
			self.first_node = node_after
		else:
			node_before.set_next_node(node_after)

		if node_after is None:
			self.last_node = node_before
		else:
			node_after.set_previous_node(node_before)

		value = node.get_value()
		if self.first_node_at_value[value] == node:
			del self.first_node_at_value[value]

			if node_after is not None and node_after.get_value() == value:
				self.first_node_at_value[value] = node_after

	def add_before(self, node, existing_node):
		node_before = existing_node.get_previous_node()
		node.set_next_node(existing_node)
		existing_node.set_previous_node(node)

		if node_before is None:
			self.first_node = node
		else:
			node_before.set_next_node(node)
			node.set_previous_node(node_before)

		value = node.get_value()
		if value not in self.first_node_at_value or self.first_node_at_value[value] == existing_node:
			self.first_node_at_value[value] = node

	def add_after(self, node, existing_node):
		node_after = existing_node.get_next_node()
		node.set_previous_node(existing_node)
		existing_node.set_next_node(node)

		if node_after is None:
			self.last_node = node
		else:
			node.set_next_node(node_after)
			node_after.set_previous_node(node)

		value = node.get_value()
		if value not in self.first_node_at_value or existing_node.get_value() < value:
			self.first_node_at_value[value] = node

	def add_item(self, item, value):
		# Replace node if it already exists with a larger value
		if item in self.item_to_node:
			if value >= self.item_to_node[item].get_value():
				return

			old_node = self.item_to_node[item]
			del self.item_to_node[item]
			self.remove_node(old_node)
		
		node = LinkedListNode(item, value)
		self.item_to_node[item] = node

		if self.first_node is None or self.last_node is None:
			self.first_node = node
			self.last_node = node
			self.first_node_at_value[value] = node
			return

		else:
			node_values = sorted([node_value for node_value in self.first_node_at_value if node_value > value])
			smallest_value_above = next((node_value for node_value in node_values), None)
			if smallest_value_above is None:
				self.add_after(node, self.last_node)
			else:
				self.add_before(node, self.first_node_at_value[smallest_value_above])

	def is_empty(self):
		return self.first_node is None

	def pop(self):
		first_node = self.first_node
		self.remove_node(first_node)
		del self.item_to_node[first_node.get_item()]

		return first_node.get_item(), first_node.get_value()

def reconstruct_path(came_from, g_score, current):
	total_path = [(current, g_score[current])]
	while current in came_from:
		current = came_from[current]
		total_path.append((current, g_score[current]))
	return total_path[::-1]


def perform_a_star(start, goal, neighbours_function, distance_function = lambda current, neighbour: 1, heuristic_function = lambda current: 0, max_search_distance=-1):
	open_set = SortedLinkedList()
	came_from = {}

	g_score = defaultdict(lambda: int(1e100))
	f_score = defaultdict(lambda: int(1e100))

	g_score[start] = 0
	f_score[start] = heuristic_function(start)
	open_set.add_item(start, f_score[start])

	while not open_set.is_empty():
		current, _ = open_set.pop()
		if current == goal or (callable(goal) and goal(current)):
			return reconstruct_path(came_from, g_score, current), {'g_score': g_score}

		for neighbour in neighbours_function(current):
			tentative_g_score = g_score[current] + distance_function(current, neighbour)
			if max_search_distance != -1 and tentative_g_score > max_search_distance:
				continue
			if tentative_g_score < g_score[neighbour]:
				came_from[neighbour] = current
				g_score[neighbour] = tentative_g_score
				current_f_score = tentative_g_score + heuristic_function(neighbour)
				f_score[neighbour] = current_f_score
				open_set.add_item(neighbour, current_f_score)

	return None, {'g_score': g_score}